In [5]:
import tensorflow as tf

import os
import sys
import pickle

import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
dir_path = os.path.split(os.getcwd())[0]
sys.path.append(dir_path)

In [3]:
from Model import data

In [4]:
DATA = data.Data()
DATA.loadDataFromJSON()

In [14]:
class ConvRNN(object):
    
    def __init__(self, batch_size, time_steps, output_size, hidden_units, vocabulary_size, layers):
        with tf.name_scope("Input"):
            self.input = tf.placeholder(tf.int32, shape=(batch_size, time_steps), name="input")
            self.labels = tf.placeholder(tf.float32, shape=(batch_size, time_steps, output_size), name="targets")
            self.mask = tf.placeholder(tf.float32, shape=(batch_size, time_steps, output_size), name="mask")
            self.seq_len = tf.placeholder(tf.int32, [batch_size,])

        with tf.name_scope("Embedding"):
            embedding_size = hidden_units
            self.embedding = tf.get_variable("embedding", (vocabulary_size, embedding_size), 
                                            initializer = tf.contrib.layers.xavier_initializer(),
                                            dtype=tf.float32) 
            #self.embedding = initializer((vocabulary_size, hidden_units))
            self.embedded_input = tf.nn.embedding_lookup(self.embedding, self.input, name="embedded_input")

        self.input_rnn = self.embedded_input
        self.input_rnn = tf.transpose(self.input_rnn)
        #self.input_rnn = tf.reshape(self.input_rnn, [batch_size, embedding_size, time_steps//3, 3])
        with tf.name_scope("RNN_conv"):
            rnn_conv_cell = lambda : tf.contrib.rnn.Conv1DLSTMCell(input_shape=[embedding_size, time_steps],
                                                                   output_channels=3,
                                                                   use_bias=True,
                                                                   kernel_shape=3)
            cells_fw = [
                    rnn_conv_cell()
                    for _ in range(layers)]
    
            cells_bw = [
                    rnn_conv_cell()
                    for _ in range(layers)]
                
            self.state = ([cell.zero_state(batch_size, dtype=tf.float32) for cell in cells_fw],
                              [cell.zero_state(batch_size, dtype=tf.float32) for cell in cells_bw])
            outputs, states_fw, states_bw = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(
                                                cells_fw = cells_fw,
                                                cells_bw=cells_bw,
                                                initial_states_fw = self.state[0],
                                                initial_states_bw = self.state[1],
                                                sequence_length = self.seq_len,
                                                dtype = tf.float32,
                                                inputs = self.input_rnn)
                
            self.next_state = (states_fw, states_bw)
            #self.outputs_rnn = tf.concat(outputs, 2)
            self.outputs_rnn = outputs
            hidden_layer_size = hidden_units*2
        

In [15]:
session=tf.InteractiveSession()
tf.reset_default_graph()
model = ConvRNN(32, 60, 2, 300, DATA.getPartition(1).vocabulary_size, 2)
print(tf.shape(model.outputs_rnn))

/home/yolkin/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


ValueError: Conv Linear expects 3D, 4D or 5D arguments: [[300, 32], [32, 300, 3]]

In [50]:
#output_size = np.max(Y_labels[:,1])+1
output_size = 2
learning_rate = 1e-1
batch_size = 32
hidden_size = 600
'''
num_features depends how do u we want to represent data
we can use w2v embbeding to send dense represetation
#num_features =embeddings.shape[1]
we can use sparse representation which for this example requires over 5000 long one hot vector
we can use sparse representation taking only instruction without arguments lowering one hot to 203 dim

in this simple model dense representation didnt show better result then filtered representation 
with only instructions
'''

#num_features = len(ByteCode.mnemonicMap)
num_features =embeddings.shape[0] + 1
num_epochs = 15

sequence_length = 100

rnn2_graph = tf.Graph()

with rnn2_graph.as_default():
    
    sequence = tf.placeholder(tf.int32,[batch_size, sequence_length])
    labels= tf.placeholder(tf.float64,[batch_size, sequence_length, output_size])
    seq_len = tf.placeholder(tf.int64, [batch_size])
    mask = tf.placeholder(tf.float64, [batch_size, sequence_length, output_size])

    cells_fw = [
        tf.nn.rnn_cell.DropoutWrapper(
        tf.contrib.rnn.BasicLSTMCell(hidden_size,activation=tf.nn.tanh),
        output_keep_prob = 0.8)
        for _ in range(1)]
    
    cells_bw = [
        tf.nn.rnn_cell.DropoutWrapper(
        tf.contrib.rnn.BasicLSTMCell(hidden_size,activation=tf.nn.tanh),
        output_keep_prob = 0.8)
        for _ in range(1)]
    
    W_embed = tf.get_variable(
         'W', shape=(num_features, hidden_size), initializer=tf.contrib.layers.xavier_initializer(), dtype=tf.float64)
    
    
    W_1 = tf.Variable(tf.random_normal([1,hidden_size*2, hidden_size*2], dtype=tf.float64))
    b_1 = tf.Variable(tf.random_normal([hidden_size*2],dtype=tf.float64))
    W_2 = tf.Variable(tf.random_normal([1,hidden_size*2, output_size], dtype=tf.float64))
    b_2 = tf.Variable(tf.random_normal([output_size], dtype=tf.float64))

    #initial_state = cell_fw.zero_state(batch_size, dtype=tf.float64)


    #outputs, state = tf.nn.dynamic_rnn(cell, sequence, 
    #        initial_state=initial_state, sequence_length=seq_len)

    embed_input = tf.nn.embedding_lookup(W_embed, sequence, name = "embedded_input")
    
    outputs, states_fw, states_bw = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(
        cells_fw = cells_fw,
        cells_bw=cells_bw,
        dtype = tf.float64,
        sequence_length = seq_len,
        inputs = embed_input)
    
    outputs = tf.concat(outputs, 2)
    
    outputs_2 = tf.nn.relu(tf.matmul(outputs, tf.tile(W_1, [tf.shape(outputs)[0],1,1])) + b_1)
    logits  = tf.matmul(outputs_2, tf.tile(W_2, [tf.shape(outputs_2)[0],1,1])) + b_2
    logits = logits*mask
    '''
    for output_batch, label_batch in zip(tf.unstack(outputs, axis=1), tf.unstack(labels, axis=1)):
        for output, label in zip(tf.unstack(output_batch, axis=0), tf.unstack(label_batch, axis=0)):
            output = tf.expand_dims(output, 0)
            label = tf.expand_dims(label, 0)
            logits = tf.matmul(output, W)+ b
            loss += tf.nn.softmax_cross_entropy_with_logits(labels=label, logits=logits)
            correct_predictions += tf.reduce_sum(tf.cast(tf.equal(tf.argmax(logits, 1), tf.argmax(label, 1)), tf.float64))
        #incorrect_prediciton += batch_size - tf.reduce_sum(tf.cast(tf.equal(tf.argmax(logits, 1), tf.argmax(label, 1)),tf.float64))
    '''
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits = logits)

    train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    #accuracy = correct_predictions/tf.reduce_sum(tf.cast(seq_len,tf.float64))
    #acc_val, acc_op = tf.metrics.accuracy(tf.argmax(labels,axis=2), tf.argmax(logits, axis=2))
    diff = batch_size*sequence_length - tf.reduce_sum(tf.cast(tf.equal(tf.argmax(labels, axis =2), tf.argmax(logits, axis=2)), tf.float32))
    nonpadsum = tf.cast(tf.reduce_sum(seq_len),tf.float32)
    accuracy = (nonpadsum- diff)/nonpadsum
    #correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))  

In [51]:
N = int(0.9*X_train.shape[0])
testN = X_train.shape[0]-N

assert N+testN == X_train.shape[0]
n_steps = N//batch_size

new_Y = []
ln, acsum = 0,0
for y_seq in Y_train:
    y_seq = list(map(lambda x: ((x[0],1) if x[1]==1 else (x[0],0)), y_seq))
    new_Y.append(y_seq)
    acsum += sum(list(map((lambda x:x[1]), y_seq)))
    ln += len(y_seq)
Y_train = new_Y
print('BASELINE:', 1-acsum/ln)
Y_train = np.array(Y_train)

from random import shuffle
indices = np.arange(X_train.shape[0])
shuffle(indices)

X_train, X_test = X_train[indices[:N]], X_train[indices[N:N+testN]]
Y_train, Y_test = Y_train[indices[:N]], Y_train[indices[N:N+testN]]

n_steps_test = int(testN/batch_size)

with tf.Session(graph=rnn2_graph) as session:
    iop = tf.global_variables_initializer()
    loc = tf.local_variables_initializer()
    session.run(iop)
    #session.run(loc)
    for epoch in range(num_epochs):
        acc_sum=0
        batch_index = 0
        for step in range(n_steps):
            cur_batch_size = 0
            batch = []
            batch_Y = []
            sequence_len = []
            while(cur_batch_size < batch_size):
                cur_seq = []
                y_cur_seq = []
                for x,y in zip(X_train[batch_index],Y_train[batch_index]):
                    if x in ByteCode.mnemonicMap:
                        cur_seq.append(x)
                        y_cur_seq.append(y)
                    if len(cur_seq) == sequence_length:
                        batch.append(cur_seq)
                        batch_Y.append(y_cur_seq)
                        sequence_len.append(len(y_cur_seq))
                        cur_batch_size += 1
                        cur_seq = []
                        y_cur_seq = []
                if cur_seq:
                    batch.append(cur_seq)
                    batch_Y.append(y_cur_seq)
                    sequence_len.append(len(y_cur_seq))
                    cur_batch_size += 1
                    batch_index += 1
            
            data = np.zeros([batch_size, sequence_length])
            filled_labels = np.zeros([batch_size, sequence_length, output_size])
            filled_labels[:,:,0] = 0
            mask_val = np.zeros([batch_size, sequence_length, output_size])
            for i, seq in enumerate(batch_Y):
                for j, label in enumerate(seq):
                    onehot=np.zeros(output_size)
                    onehot[label[1]] = 1
                    filled_labels[i,j,:] = onehot
                    mask_val[i,j,:] = 1
            for i in range(batch_size):
                for j in range(sequence_len[i]):
                    data[i,j] = dictionary.get(batch[i][j],0)+1
            
            feed = {sequence:data, labels:filled_labels, seq_len:sequence_len, mask:mask_val}
            ops = [logits, diff, accuracy, train_step]
            logits_val, diff_val, acc_fixed, _= session.run(ops, feed_dict=feed)
            #acc_fixed = session.run([accuracy], feed_dict={seq_len:sequence_len})
            #session.run(loc)
            
            #print(acc_fixed)
            #print(diff_val)
            #print(np.sum(filled_labels))
            #print(np.sum(sequence_len))
            #print(np.argmax(filled_labels,axis=2)[0][:sequence_len[0]])
            #print(np.argmax(logits_val,axis=2)[0][:sequence_len[0]])
            '''
            zum = 0
            for i,sl in enumerate(sequence_len):
                zum += np.sum(np.argmax(logits_val[i,sl:,:],axis=1))
            print(zum)
            '''
            #print()

        acc_sum=0
        batch_index = 0
        for step in range(n_steps_test):
            cur_batch_size = 0
            batch = []
            batch_Y = []
            sequence_len = []
            while(cur_batch_size < batch_size):
                cur_seq = []
                y_cur_seq = []
                for x,y in zip(X_test[batch_index],Y_test[batch_index]):
                    if x in ByteCode.mnemonicMap:
                        cur_seq.append(x)
                        y_cur_seq.append(y)
                    if len(cur_seq) == sequence_length:
                        batch.append(cur_seq)
                        batch_Y.append(y_cur_seq)
                        sequence_len.append(len(y_cur_seq))
                        cur_batch_size += 1
                        cur_seq = []
                        y_cur_seq = []
                if cur_seq:
                    batch.append(cur_seq)
                    batch_Y.append(y_cur_seq)
                    sequence_len.append(len(y_cur_seq))
                    cur_batch_size += 1
                    batch_index += 1
            
            data = np.zeros([batch_size, sequence_length])
            filled_labels = np.zeros([batch_size, sequence_length, output_size])
            filled_labels[:,:,0] = 0
            mask_val = np.zeros([batch_size, sequence_length, output_size])
            for i, seq in enumerate(batch_Y):
                for j, label in enumerate(seq):
                    onehot=np.zeros(output_size)
                    onehot[label[1]] = 1
                    filled_labels[i,j,:] = onehot
                    mask_val[i,j,:] = 1
            for i in range(batch_size):
                for j in range(sequence_len[i]):
                    data[i,j] = dictionary.get(batch[i][j],0)+1
            
            feed = {sequence:data, labels:filled_labels, seq_len:sequence_len, mask:mask_val}
            ops = [logits, diff, accuracy]
            logits_val, diff_val, acc_fixed = session.run(ops, feed_dict=feed)
            acc_sum += acc_fixed
            
        print("Epoch %d, step %d ,acc: %f" % (epoch, step, acc_sum/n_steps_test))
        index = np.argmax(np.sum(filled_labels,axis=(1,2)))
        print(batch[index])
        print(np.argmax(filled_labels,axis=2)[index][:sequence_len[index]])
        print(np.argmax(logits_val,axis=2)[index][:sequence_len[index]])

4993
555
5548
BASELINE: 0.9123273159846137
Epoch 0, step 16 ,acc: 0.888588
['lload_0', 'lload_2', 'ladd', 'lstore', 'lload_0', 'lload_2', 'lxor', 'lconst_0', 'lcmp', 'iflt', 'lload', 'lload_2', 'lxor', 'lconst_0', 'lcmp', 'ifge', 'new', 'dup', 'getstatic', 'iconst_2', 'anewarray', 'dup', 'iconst_0', 'lload_0', 'invokestatic', 'aastore', 'dup', 'iconst_1', 'lload_2', 'invokestatic', 'aastore', 'invokespecial', 'athrow', 'lload', 'lreturn']
[0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Epoch 1, step 16 ,acc: 0.914673
['lload_0', 'lload_2', 'ladd', 'lstore', 'lload_0', 'lload_2', 'lxor', 'lconst_0', 'lcmp', 'iflt', 'lload', 'lload_2', 'lxor', 'lconst_0', 'lcmp', 'ifge', 'new', 'dup', 'getstatic', 'iconst_2', 'anewarray', 'dup', 'iconst_0', 'lload_0', 'invokestatic', 'aastore', 'dup', 'iconst_1', 'lload_2', 'invokestatic', 'aastore', 'invokespecial', 'athrow', 'lload', 'lreturn']
[0 0 0 0 1 1 